# EM-алгоритм для смеси бернуллиевских случайных величин

<a id='toc'></a>
# Содержание
* [1. Теория](#em_algo_bern_theory)
    * [1.1 Формулировка задачи](#bern_task)
    * [1.2 Решение](#bern_sol)
* [2. Реализация](#em_algo_bern_impl)
* [3. Применение](#em_algo_bern_usage)

In [ ]:
%autosave 30

import numpy as np
import pandas as pd
import copy
import sys
_add_to_path = True
import math
import scipy
import pickle as pkl
from scipy import stats
from scipy.special import erfc

from itertools import product, chain
from collections import defaultdict, Counter
from tqdm import tqdm

#matplotlib
import matplotlib
import matplotlib as mp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import PolyCollection
from matplotlib.colors import colorConverter
%matplotlib inline

#sklearn
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.feature_extraction.text import CountVectorizer

matplotlib.rcParams['legend.markerscale'] = 1.5     # the relative size of legend markers vs. original
matplotlib.rcParams['legend.handletextpad'] = 0.01
matplotlib.rcParams['legend.labelspacing'] = 0.4    # the vertical space between the legend entries in fraction of fontsize
matplotlib.rcParams['legend.borderpad'] = 0.5       # border whitespace in fontsize units
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams['font.serif'] = 'Times New Roman'
matplotlib.rcParams['axes.labelsize'] = 20
matplotlib.rcParams['axes.titlesize'] = 20

matplotlib.rc('xtick', labelsize=14)
matplotlib.rc('ytick', labelsize=14)
matplotlib.rc('legend', fontsize=16)

matplotlib.rc('font', **{'family':'serif'})
matplotlib.rc('text', usetex=True)
matplotlib.rc('text.latex', unicode=True)
matplotlib.rc('text.latex', preamble=r'\usepackage[utf8]{inputenc}')
matplotlib.rc('text.latex', preamble=r'\usepackage[english]{babel}') 
matplotlib.rcParams['axes.labelsize'] = 20

In [ ]:
if _add_to_path:
    sys.path.append('../../')
from ml.core import Checker, Printer
from ml.mixtures import *

%load_ext autoreload
%autoreload 1
# Below comes the list of modules which is automatically reimported

$\newcommand{\Sum}{\sum\limits}$
$\newcommand{\Prod}{\prod\limits}$
$\newcommand{\Max}{\max\limits}$
$\newcommand{\Min}{\min\limits}$
$\newcommand{\Int}{\int\limits}$
$\newcommand{\Exp}{\mathbb{E}}$
$\newcommand{\Var}{\mathbb{V}}$
$\newcommand{\Energy}{\mathcal{E}}$
$\newcommand{\Prob}{\mathcal{P}}$
$\newcommand{\N}{\mathcal{N}}$


$\newcommand{\LogLike}{\mathcal{L}}$
$\newcommand{\Like}{\ell}$

$\newcommand{\bolda}{\boldsymbol{a}}$
$\newcommand{\boldA}{\boldsymbol{A}}$
$\newcommand{\ba}{\bolda}$
$\newcommand{\bA}{\boldA}$

$\newcommand{\boldb}{\boldsymbol{b}}$
$\newcommand{\boldB}{\boldsymbol{B}}$
$\newcommand{\bb}{\boldb}$
$\newcommand{\bB}{\boldB}$

$\newcommand{\boldd}{\boldsymbol{d}}$
$\newcommand{\boldD}{\boldsymbol{D}}$
$\newcommand{\bd}{\boldd}$
$\newcommand{\bD}{\boldD}$

$\newcommand{\boldf}{\boldsymbol{f}}$
$\newcommand{\boldF}{\boldsymbol{F}}$
$\newcommand{\bf}{\boldf}$
$\newcommand{\bF}{\boldF}$

$\newcommand{\boldt}{\boldsymbol{t}}$
$\newcommand{\boldT}{\boldsymbol{T}}$
$\newcommand{\bt}{\boldsymbol{\boldt}}$
$\newcommand{\bT}{\boldsymbol{\boldT}}$

$\newcommand{\boldx}{\boldsymbol{x}}$
$\newcommand{\boldX}{\boldsymbol{X}}$
$\newcommand{\bx}{\boldx}$
$\newcommand{\bX}{\boldX}$

$\newcommand{\boldY}{\boldsymbol{Y}}$
$\newcommand{\boldy}{\boldsymbol{y}}$
$\newcommand{\bY}{\boldY}$
$\newcommand{\by}{\boldy}$

$\newcommand{\boldZ}{\boldsymbol{Z}}$
$\newcommand{\boldz}{\boldsymbol{z}}$
$\newcommand{\bZ}{\boldZ}$
$\newcommand{\bz}{\boldz}$

$\newcommand{\boldTheta}{\boldsymbol{\Theta}}$
$\newcommand{\boldtheta}{\boldsymbol{\theta}}$
$\newcommand{\bTheta}{\boldTheta}$
$\newcommand{\btheta}{\boldtheta}$

<a id='em_algo_bern_theory'></a>
# 1. Теория [[toc](#toc)]

![Templates](https://www.dropbox.com/s/lx47mcsq3yyhmk2/templates.png?dl=1)

<a id='bern_task'></a>
## 1.1 Формулировка задачи [[toc](#toc)]

Пусть всего у нас $K$ шаблонов, то есть $K$ компонент смеси. Каждый объект выборки $-$ это $D$-мерный бинарный вектор: $X = \{ \mathbf{x}_i \}_{i=1}^N, \mathbf{x}_i \in \{ 0, 1 \}^D$. Каждое из $K$ бинарных распределений задает вероятность признака $d$ принимать значение 1, обозначаемое ${\theta}_{kd}$. Соответственно вероятность признака $d$ быть равным 0 есть $1 - {\theta}_{kd}$. Таким образом, правдоподобие объекта $\mathbf{x}_i$ при условии принадлежности распределению $k$ есть:
$$
	p(\mathbf{x}_i | t_i = k, \theta) = \prod_{d=1}^D \theta_{kd}^{x_{id}} (1 - \theta_{kd})^{1 - x_{id}},
$$
где $t_i$ - скрытая переменная, принимающая значения от $1$ до $K$, кодирующие к какому распределению относится объект. Априорное распределение на $t_i$ будем полагать равномерным: $p(t_i = k) = 1 / K$.

EM-алгоритм итеративно оптимизирует логарифм неполного правдоподобия:
$$
	\log p(X | \theta) = \sum_{i=1}^N \log \sum_{k = 1}^K p(\mathbf{x}_i | t_i = k, \theta) - N \log K
$$
На E-шаге рассчитывается апостериорное распределение на скрытые переменные при старых значениях параметров:
$$
	p(T | X, \theta^{old}) = \frac{p(T) p(X | T, \theta^{old})}{p(X | \theta^{old})}
$$
В данном случае апостериорные распределения для каждого объекта независимы, то есть, $p(T | X, \theta) = \prod_{i=1}^N p(t_i | \mathbf{x}_i, \theta)$.

Затем, на M-шаге выполняется оптимизиация по параметрам распределений:
$$
	\theta^{new} = \arg \max_{\theta} \mathbb{E}_{p(T | X, \theta^{old})} \log p(X, T | \theta) 
$$
В данном случае максимум можно найти, приравняв производную по каждому из параметров к нулю.

Эти два шага чередуются необходимое число итераций.

<a id='bern_sol'></a>
## 1.2 Формулировка задачи [[toc](#toc)]

Выпишите конечные формулы для апостериорных распределений $p(t_i | \mathbf{x}_i, \theta)$. Найдите оценки для параметров, получаемые на M-шаге.

$$
p(\boldx_n, t_n=k;\boldtheta) = \pi_k p(\boldx_n|t_n=k;\boldtheta) = \pi_k\Prod_{d=1}^D \theta_{kd}^{x_{nd}}(1-\theta_{kd})^{1 - x_{nd}}.
$$

$$
\gamma_{nk} = p(t_n = k|\boldx_n, \boldtheta) = \frac{p(\boldx_n, t_n; \boldtheta)}{p(\boldx_n; \boldtheta)} = \frac{\pi_k \cdot p(\boldx_n| t_n=k; \boldtheta)}{\Sum_{i=1}^K \pi_i \cdot p(\boldx_n| t_n = i; \boldtheta)} = 
\frac{\pi_k \cdot \Prod_{d=1}^D \theta_{kd}^{x_{nd}}(1 - \theta_{kd})^{1-x_{nd}}}{\Sum_{i=1}^K \pi_i \cdot \Prod_{d=1}^D \theta_{id}^{x_{nd}}(1 - \theta_{id})^{1-x_{nd}}}.
$$

На M-шаге требуется найти максимум ожидания правдоподобия
\begin{gather}
\Exp\LogLike = \Exp_{T \backsim p(T|X;\boldtheta)} \LogLike(X, T; \theta) = \Sum_{n=1}^N \Exp_{t \backsim p(t | \boldx_n;\boldtheta)} \log p(\boldx_n, t_n; \boldtheta) = \Sum_{n=1}^N \Sum_{k=1}^K p(t_n=k|\boldx_n; \boldtheta) \log p(\boldx_n, t_n = k; \boldtheta) = \\ 
= \Sum_{n=1}^N\Sum_{k=1}^K \gamma_{nk} \left(\log \pi_k + \Sum_{d=1}^D \left\{x_{nd} \log \theta_{kd} + (1 - x_{nd}) \log (1 - \theta_{nd})\right\} \right)
\end{gather}

$$
\frac{\partial\Exp\LogLike}{\partial \pi_k} = \frac{\Sum_{n=1}^N \gamma_{nk}}{\pi_k} - \lambda = 0 \Rightarrow \pi_k \lambda = \Sum_{n=1}^N \gamma_{nk} \Rightarrow \lambda = N, \pi_k = \frac{\Sum_{n=1}^N \gamma_{nk}}{N} = \frac{N_k}{N}, \text{ где } N_k = \Sum_{n=1}^N \gamma_{nk}.
$$

$$
\frac{\partial\Exp\LogLike}{\partial \theta_{kd}} = \Sum_{n=1}^N \gamma_{nk} \left(\frac{x_{nd}}{\theta_{kd}} - \frac{1 - x_{nd}}{1 - \theta_{kd}}\right) = \Sum_{n=1}^N \gamma_{nk} \frac{x_{nd}(1 - \theta_{kd}) - (1 - x_{nd})\theta_{kd}}{\theta_{kd}(1 - \theta_{kd})} = \Sum_{n=1}^N \gamma_{nk} \frac{x_{nd}- \theta_{kd}}{\theta_{kd}(1 - \theta_{kd})} \Rightarrow \theta_{kd} = \frac{ \Sum_{n=1}^N \gamma_{nk} x_{nd}}{ \Sum_{n=1}^N \gamma_{nk}} = 
\frac{ \Sum_{n=1}^N \gamma_{nk} x_{nd}}{N_k}
$$

В случае $\pi_k = 1 / K$ получаем **EM-алгоритм**:

* E-шаг

$$
\gamma_{nk} = p(t_n = k|\boldx_n, \boldtheta) = \frac{\Prod_{d=1}^D \theta_{kd}^{x_{nd}}(1 - \theta_{kd})^{1-x_{nd}}}{\Sum_{i=1}^K \Prod_{d=1}^D \theta_{id}^{x_{nd}}(1 - \theta_{id})^{1-x_{nd}}}.
$$

* M-шаг

$$\theta_{kd} = \frac{ \Sum_{n=1}^N \gamma_{nk} x_{nd}}{ \Sum_{n=1}^N \gamma_{nk}} = 
\frac{ \Sum_{n=1}^N \gamma_{nk} x_{nd}}{N_k}$$

Запустите EM-алгоритм на изображениях цифр 6 и 9 для $K=2$, сделайте 30 итераций. Постройте график логарифма правдоподобия в зависимости от числа итераций, а также визуализируйте шаблоны, полученные после пересчета на каждой итерации. Удалось ли вам получить шаблоны этих цифр?

<a id='em_algo_bern_impl'></a>
# 2. Реализация [[toc](#toc)]

In [ ]:
class EMBernoulli:
    def __init__(self, n_components, max_iter=10, loglikelihoods=True, verbose=False, eps=1e-40, 
                 omit_constants=True, batch_size=None, memory_limit=512):
        """
        Аргументы:
            param: eps
            param: omit_constants   - 
            param: batch_size       - 
            param: memory_limit - 
            
        """
        self._n_components = n_components
        self._max_iter = max_iter
        self._loglikelihoods = loglikelihoods
        self._verbose = verbose
        self._eps = eps
        self._omit_constants = omit_constants
        self._batch_size = batch_size
        self._memory_limit = memory_limit
        
    def get_thetas(self):
        if self._omit_constants:
            const_values = self.maximums[self.equal_columns]
            thetas = np.zeros((self._n_components, self._n_dim + len(const_values)), dtype=np.float64)
            thetas[:, self.equal_columns] = const_values[None, :]
            thetas[:, self.nequal_columns] = self.T
        else:
            thetas = self.T
        return np.array(thetas)
    def get_gammas(self):
        return np.array(self.G.T)

    def __call__(self, X):
        self._initialize(X)
        while (not self._stop()):
            self._E_step()
            self._M_step()
            self._print('n_iter = {}: likelihood = {}'.format(self._n_iter,  self._likelihoods[-1]))
            self._n_iter += 1
            
    def _initialize(self, X):
        X = np.array(X)
        if self._omit_constants:
            self.maximums = np.max(X, axis=0)
            self.minimums = np.min(X, axis=0) 
            self.equal_columns = np.where(self.maximums == self.minimums)[0]
            self.nequal_columns = np.where(self.maximums != self.minimums)[0]
            self.X  = X[:, self.nequal_columns]      # [N x D]
        else:
            self.X = X                               # [N x D]            
        self.Xt = self.X.T                           # [D x N]
        self._n_samples, self._n_dim = self.X.shape
        
        self._find_batch_size()
        
        self.G = np.random.uniform(size=(self._n_components, self._n_samples)) # [K x N]
        self.G = np.maximum(self.G, self._eps)
        self.G /= np.sum(self.G, axis=0)[None, :]
        self.T = np.zeros((self._n_components, self._n_dim))                   # [K x D]
        
        self._M_step()
        self._n_iter = 0
        self._likelihoods = []
        
    def _find_batch_size(self):
        if self._batch_size is not None:
            return
        full_size = (1.0 * self._n_samples * self._n_dim * self._n_components * self.X.dtype.type(0).nbytes) / (2 ** 20)
        if full_size > self._memory_limit:
            alpha = 1.0 * full_size / self._memory_limit
            self._batch_size = int(self._n_samples / alpha + 0.5)
        else:
            self._batch_size = self._n_samples
        new_full_size = (1.0 * self._batch_size * self._n_dim * self._n_components * self.X.dtype.type(0).nbytes) / (2 ** 20)
        self._print('Found batch size = {} for memory limit = {}'.format(self._batch_size, self._memory_limit))
        self._print('\tFull size = {}, new full size = {}'.format(int(full_size), int(new_full_size)))
        
    def _E_step(self):
        T = self.T.reshape((self._n_components, -1, self._n_dim))                # [K x 1 x D]
        if self._batch_size == 1:
            likelihood = 0
            for n_sample in range(self._n_samples):
                x = self.X[[n_sample], :]                                        # [1 x D]
                logs = np.sum(x * np.log(self.T) + (1 - x) * np.log(1 - self.T), axis=1) # [K x D] -> [K]
                likelihood += np.sum(self.G[:, n_sample] * logs)
                self.G[:, n_sample] = self._softmax_vector(logs)
            self._likelihoods.append(likelihood)
            return
        if self._batch_size < self._n_samples:
            Ss = []
            for i in range(0, self._n_samples, self._batch_size):
                left, right = i, min(i + self._batch_size, self._n_samples)
                X = self.X[left:right].reshape((-1, right - left, self._n_dim))  # [1 x B x D]
                S = np.sum(X * np.log(T) + (1 - X) * np.log(1 - T), axis=2)      # [K x B x D] -> [K x B]
                Ss.append(S)                                                     # [K x N]
            S = np.concatenate(Ss, axis=1)
            assert S.shape == (self._n_components, self._n_samples)
        elif self._batch_size == self._n_samples:
            X = self.X.reshape((-1, self._n_samples, self._n_dim))               # [1 x N x D]
            S = np.sum(X * np.log(T) + (1 - X) * np.log(1 - T), axis=2)          # [K x N x D] -> [K x N]
            assert S.shape == (self._n_components, self._n_samples)
        else:
            assert False, "Something is wrong!"
        likelihood = np.sum(np.multiply(self.G, S))
        self.G = self._softmax_column(S)
        self._likelihoods.append(likelihood)

    def _M_step(self):
        if self._batch_size < self._n_samples:
            for n_component in range(self._n_components):
                weights = self.G[[n_component], :]                               # [1 x N]
                self.T[n_component, :] = np.sum(weights * self.Xt, axis=1) / np.sum(weights)
        elif self._batch_size == self._n_samples:
            G = self.G.reshape((self._n_components, 1, self._n_samples))
            X = self.Xt.reshape((1, self._n_dim, self._n_samples))
            self.T = np.sum(G * X, axis=2) / np.sum(self.G, axis=1)[:, None]
        else:
            assert False, "Something is wrong!"
        self.T = np.maximum(self.T, self._eps)
        self.T = np.minimum(self.T, 1 - self._eps)

    def _stop(self):
        if self._n_iter >= self._max_iter:
            return True
        return False

    def _softmax_vector(self, x):
        max_prob = np.max(x)
        x -= max_prob
        np.exp(x, x)
        sum_prob = np.sum(x)
        x /= sum_prob
        return x
    
    def _softmax_column(self, X):
        max_prob = np.max(X, axis=0)[None, :]
        X -= max_prob
        np.exp(X, X)
        sum_prob = np.sum(X, axis=0)[None, :]
        X /= sum_prob
        return X

    def _print(self, msg):
        if self._verbose:
            print(msg)

<a id='em_algo_bern_usage'></a>
# 3. Применение [[toc](#toc)]
* [Загрузка данных](#em_algo_bern_load)
* [K = 10](#em_algo_bern_K10)
* [K = 15](#em_algo_bern_K15)
* [K = 20](#em_algo_bern_K20)
* [K = 25](#em_algo_bern_K25)
* [K = 5 per digit](#em_algo_bern_K5_pd)

<a id=em_algo_bern_load></a>
## 3.1 Загрузка данных [[toc](#toc)]

Загрузите обучающую и тестовую выборки [здесь](https://www.dropbox.com/s/8092jukwxapc04o/mnist.zip?dl=0). Первый столбец $-$ это метка цифры, изображенной на данной картинке. Бинаризуйте все изображения по порогу 127.

In [ ]:
digits = pd.read_csv('./mnist/mnist_train.csv', header=None)
digits = digits.values
labels = digits[:, 0]
digits = digits[:, 1:]
digits = (digits > 127).astype(np.int32)
n_samples, n_dim = labels.shape, digits.shape
print(n_samples, n_dim)

<a id=em_algo_bern_Ks></a>
### Анализ при различном числе компонент [[toc](#toc)]

In [ ]:
active_labels = [6, 9]
mask = np.zeros(n_samples, dtype=np.int32)
for label in active_labels:
    mask += (labels == label)
mask = mask > 0
X = digits[mask]
print(X.shape)

em = EMBernoulli(n_components=2, max_iter=60, batch_size=None, verbose=True)
em(X)

In [ ]:
thetas = em.get_thetas()
thetas = thetas.reshape((2, 28, 28))
fsize = 6
fig, axes = plt.subplots(1, 2, figsize=(2 * fsize, fsize))
axes[0].imshow(thetas[0], cmap='afmhot', vmin=-0.2, interpolation='none')
axes[1].imshow(thetas[1], cmap='afmhot', vmin=-0.2, interpolation='none')

<a id=em_algo_bern_Ks></a>
#### Анализ при различном числе компонент [[toc](#toc)]

Выполните аналогичное исследование для всех изображений всех цифр. Используйте 50 итераций и разные значения $K=10, 15, 20$, для каждого визуализируйте логарифм неполного правдоподобия и получаемые шаблоны после каждой итерации EM-алгоритма. Если ваша реализация работает слишком медленно, отберите некоторое количество изображений каждой цифры из всей выборки и работайте только с ними. 

In [ ]:
def find_templates(digits, labels, n_components, percentage=1.0, 
                   memory_limit=512, max_iter=50, em_verbose=True, batch_size=None, random_state=1234):
    if percentage < 1.0:
        train_digits, val_digits, train_labels, val_labels = train_test_split(digits, labels, train_size=percentage, 
                                                                              stratify=labels)
    else:
        train_digits, train_labels = digits, labels
    print(train_digits.shape, train_labels.shape)

    em = EMBernoulli(n_components=n_components, batch_size=batch_size, 
                     memory_limit=memory_limit, max_iter=max_iter, verbose=em_verbose)
    em(train_digits)
    thetas = em.get_thetas()
    pkl.dump(thetas, open('K{}_templates.pkl'.format(n_components), 'wb'))
    pkl.dump(em.get_gammas(), open('K{}_gammas.pkl'.format(n_components), 'wb'))

In [ ]:
def plot_templates(n_components, fsize=2, n_cols=5):
    thetas = pkl.load(open('K{}_templates.pkl'.format(n_components), 'rb'))
    thetas.shape = (n_components, 28, 28)
    n_rows = int(np.ceil(n_components / n_cols))
    fig, axarr = plt.subplots(n_rows, n_cols, figsize=(n_cols * fsize, n_rows * fsize))
    for n_digit, (n_row, n_col) in enumerate(product(range(n_rows), range(n_cols))):
        ax = axarr[n_row][n_col]
        ax.imshow(thetas[n_digit], cmap='afmhot', vmin=-0.2, interpolation='none')
        ax.set_title('{}'.format(n_digit))
        ax.set_xticks([])
        ax.set_yticks([])
    plt.tight_layout()

<a id=em_algo_bern_K10></a>
#### K = 10 [[toc](#toc)]

In [ ]:
percentage = 1.0
n_components = 10
find_templates(digits, labels, n_components, percentage, 50)

In [ ]:
plot_templates(n_components)

<a id=em_algo_bern_K15></a>
#### K = 15 [[toc](#toc)]

In [ ]:
percentage = 1.0
n_components = 15
find_templates(digits, labels, n_components, percentage, 50)

In [ ]:
plot_templates(n_components)

<a id=em_algo_bern_K20></a>
#### K = 20 [[toc](#toc)]

In [ ]:
percentage = 1.0
n_components = 20
find_templates(digits, labels, n_components, percentage, batch_size=None, memory_limit=256, max_iter=50)

In [ ]:
plot_templates(n_components)

In [ ]:
mapping = {0: 6, 1: 4, 2: 0, 3: 7, 4: 8,
           5: 3, 6: 1, 7: 6, 8: 2, 9: 3,
           10: 5, 11: 8, 12: 7, 13: 6, 14: 9,
           15: 4, 16: 1, 17: 2, 18: 0, 19: 5}
pred_labels = np.array([mapping[label] for label in np.argmax(gammas, axis=1)])
print('accuracy = ', np.mean(pred_labels == train_labels))

<a id=em_algo_bern_K25></a>
#### K = 25 [[toc](#toc)]

In [ ]:
percentage = 1.0
n_components = 25
find_templates(digits, labels, n_components, percentage=percentage, max_iter=50, memory_limit=256)

In [ ]:
plot_templates(n_components)

In [ ]:
print('K =', n_components)
gammas = pkl.load(open('K{}_gammas.pkl'.format(n_components), 'rb'))
candidates = [[9, 7], [7],    [9, 7], [6   ], [0],
              [3, 5], [1],    [8],    [4, 9], [0],
              [6],    [1],    [2],    [3, 8], [1],
              [4, 9], [3, 8], [6],    [5],    [4, 9],
              [2],    [8],    [4, 9], [7],    [3, 8]]
n_steps = 10
best_accuracy = 0
best_mapping = None
for values in tqdm(product(*candidates)):
    mapping = {label:digit for label, digit in enumerate(values)}
    pred_labels = np.array([mapping[label] for label in np.argmax(gammas, axis=1)])
    accuracy = np.mean(pred_labels == labels)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_mapping = copy.deepcopy(mapping)
        print('accuracy = ', accuracy)

<a id=em_algo_bern_K5_pd></a>
#### K = 5 per digit [[toc](#toc)]

In [ ]:
n_components = 5
for digit in range(10):
    X = digits[labels == digit]
    print('digit = {}: n_samples = {}'.format(digit, X.shape[0]))
    em = EMBernoulli(n_components, max_iter=50, verbose=False)
    em(X)
    gammas = em.get_gammas()
    thetas = em.get_thetas()
    pkl.dump(thetas, open('K{}_D{}_thetas.pkl'.format(n_components, digit), 'wb'))
    pkl.dump(gammas, open('K{}_D{}_gammas.pkl'.format(n_components, digit), 'wb'))

In [ ]:
n_components = 5
templates = []
for digit in range(10):
    templates.append(pkl.load(open('K{}_D{}_thetas.pkl'.format(n_components, digit), 'rb')))
templates = np.vstack(templates)
templates = np.maximum(templates, 1e-100)
templates = np.minimum(templates, 1 - 1e-100)
print(templates.shape)

In [ ]:
n_templates = 50
n_cols = 10
n_rows = int(1.0 * n_templates / n_cols + 0.5)
fsize = 2
f, axarr = plt.subplots(n_rows, n_cols, figsize=(n_cols * fsize, n_rows * fsize))
for n_label, (n_row, n_col) in enumerate(product(range(n_rows), range(n_cols))):
    axarr[n_row][n_col].imshow(templates[n_label].reshape((28, 28)), cmap='afmhot')
    axarr[n_row][n_col].set_xticks([])
    axarr[n_row][n_col].set_yticks([])
plt.tight_layout()

In [ ]:
X = digits
y = labels

n_correct = 0
period = 1000
for n_sample in range(X.shape[0]):
    x = X[[n_sample], :]
    plt.imshow(x.reshape((28, 28)))
    plt.title('{}'.format(y[n_sample]))
    logs = np.sum(x * np.log(templates) + (1 - x) * np.log(1 - templates), axis=1)
    pred_y = np.argmax(logs) // n_components
    if pred_y == y[n_sample]:
        n_correct += 1
    if (n_sample + 1) % period == 0:
        print('accuracy =', n_correct / (n_sample + 1))
accuracy = n_correct / len(y)
print('accuracy = {}'.format(accuracy))

Для каких значений $K$ вам удалось выделить шаблоны всех цифр? Какие цифры оказались самыми сложными для распознавания и потребовали нескольких шаблонов?

Для одной из обученных моделей проведите следующее исследование. Вручную свяжите каждый шаблон с цифрой, которая на нем изображена. Затем, используя апостериорные распределения объектов тестовой выборки и привязку шаблонов к цифрам, определите цифру каждого тестового изображения и подсчитайте точность классификации. Согласуется ли правдоподобие обучающей и тестовой выборок с точностью классификации на тестовой выборке? 